In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import datetime
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import plotly.graph_objs as go

1) Download 5-minute timeframe data for top 3 constituents of Banknifty
index from yfinance for last 59 days.

According to NSE website, top 3 constituents of Banknifty are HDFCBANK, ICICIBANK and KOTAKBANK

In [ ]:
constituents = ["HDFCBANK.NS", "ICICIBANK.NS", "KOTAKBANK.NS"]
index = "^NSEBANK"

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=59)

def download_data(ticker):
    data = yf.download(ticker, start=start_date, end=end_date, interval="5m")
    return data

# data download
banknifty_data = download_data(index)
constituent_data = {ticker: download_data(ticker) for ticker in constituents}

print("BankNifty Data:")
print(banknifty_data.head())

for ticker, data in constituent_data.items():
    print(f"\n{ticker} Data:")
    print(data.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
BankNifty Data:
                                   Open          High           Low  \
Datetime                                                              
2023-10-03 09:15:00+05:30  44561.500000  44561.500000  44316.250000   
2023-10-03 09:20:00+05:30  44325.851562  44343.000000  44291.601562   
2023-10-03 09:25:00+05:30  44287.449219  44295.601562  44264.449219   
2023-10-03 09:30:00+05:30  44270.199219  44311.648438  44247.449219   
2023-10-03 09:35:00+05:30  44281.148438  44326.050781  44276.500000   

                                  Close     Adj Close  Volume  
Datetime                                                       
2023-10-03 09:15:00+05:30  44328.648438  44328.648438       0  
2023-10-03 09:20:0

2) Run a rolling multiple regression among banknifty and the 3 constituents
while making banknifty as y and rest 3 constituents as x variable for
regression.
3) Make a rolling window variable and choose any window length you like
for regression.

-> If banknifty = y and rest 3 constituents as x1, x2, x3
              then y = m1*x1 + m2*x2 + m3*x3 + constant where
              m1, m2, m3 will become the rolling constituents' coefficients

In [ ]:
y = banknifty_data['Close']
X = pd.DataFrame({ticker: data['Close'] for ticker, data in constituent_data.items()})
X = sm.add_constant(X)  # Add a constant term to the predictor

# rolling window size
window_size = 20

# rolling regression
model = RollingOLS(y, X, window=window_size)
rolling_results = model.fit()

# Extracting the rolling coefficients
coefficients = rolling_results.params.copy()
predicted_values = (X * coefficients).sum(axis=1)

residuals = y - predicted_values

print(rolling_results.params.tail())


                                 const  HDFCBANK.NS  ICICIBANK.NS  \
Datetime                                                            
2023-11-30 10:40:00+05:30  1660.699562     2.294541     16.726424   
2023-11-30 10:45:00+05:30   856.582161     1.913743     19.500127   
2023-11-30 10:50:00+05:30  2469.072590     4.141652     20.999533   
2023-11-30 10:55:00+05:30   428.411598     5.004868     21.759334   
2023-11-30 11:00:00+05:30 -2620.545652     6.534461     21.789231   

                           KOTAKBANK.NS  
Datetime                                 
2023-11-30 10:40:00+05:30     13.483522  
2023-11-30 10:45:00+05:30     12.802861  
2023-11-30 10:50:00+05:30      9.114288  
2023-11-30 10:55:00+05:30      9.111847  
2023-11-30 11:00:00+05:30      9.484057  


4) You have the rolling residuals from the regressions, now standardize
them and from here we will call standardized residuals as ‘z-score’.

In [ ]:
# rolling mean and standard deviation of residuals
rolling_mean = residuals.rolling(window=window_size).mean()
rolling_std = residuals.rolling(window=window_size).std()

# Standardizing to get z-scores
z_scores = (residuals - rolling_mean) / rolling_std

z_scores


Datetime
2023-10-03 09:15:00+05:30         NaN
2023-10-03 09:20:00+05:30         NaN
2023-10-03 09:25:00+05:30         NaN
2023-10-03 09:30:00+05:30         NaN
2023-10-03 09:35:00+05:30         NaN
                               ...   
2023-11-30 10:40:00+05:30   -0.351288
2023-11-30 10:45:00+05:30   -0.340661
2023-11-30 10:50:00+05:30   -0.241354
2023-11-30 10:55:00+05:30   -0.564789
2023-11-30 11:00:00+05:30   -0.735294
Length: 2947, dtype: float64

ploting Z-Scores Over Time

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming 'z_scores' is a Pandas Series with a DateTimeIndex
# Convert 'z_scores' to a DataFrame for Plotly
z_scores_df = z_scores.reset_index()
z_scores_df.columns = ['Date', 'Z-Score']

# Create an interactive line plot
fig = px.line(z_scores_df, x='Date', y='Z-Score', title='Z-Scores Over Time')

# Add dotted lines at z_scores = -2 and z_scores = 2
fig.add_hline(y=2, line_dash="dot", line_color="green", annotation_text="z_score = 2", annotation_position="bottom right")
fig.add_hline(y=-2, line_dash="dot", line_color="red", annotation_text="z_score = -2", annotation_position="top right")

# Show the plot
fig.show()


5. **Trading Strategy when Z-Score >= 2.0**:
   a. Short y and go long on x (short banknifty and buy the 3 constituents in the ratio of their coefficients from the regression)
   b. Square-off the positions when z-score <= 0

6. **Trading Strategy when Z-Score <= -2.0**:
   a. Long on y and short on x (buy banknifty and short the 3 constituents in the ratio of their coefficients from the regression)
   b. Square-off the positions once z-score >= 0



- Accounted for pyramiding

In [ ]:
# Initialize variables for the trading simulation
initial_balance = 800000 # asumming $10000 = 10000*80 RS
balance = initial_balance
positions = {'BankNifty': 0, 'HDFCBANK.NS': 0, 'ICICIBANK.NS': 0, 'KOTAKBANK.NS': 0}
tradebook = []
prev_z_score = None  # Variable to hold the previous z-score

# Function to calculate the number of shares to buy/sell based on allocation and stock price
def calculate_shares(price, allocation):
    return int(allocation / price)

# Iterate through the data
for date, current_z_score in z_scores.items():
    next_open_prices = banknifty_data['Open'].get(date, None)
    next_open_prices_constituents = {ticker: constituent_data[ticker]['Open'].get(date, None) for ticker in constituents}

    if None in next_open_prices_constituents.values() or next_open_prices is None:
        continue  # Skip if next open price data is missing

    total_coeff = sum(abs(coefficients[ticker][-1]) for ticker in constituents)
    allocations = {ticker: (abs(coefficients[ticker][-1]) / total_coeff) * (balance/2) for ticker in constituents}

    if current_z_score >= 2.0 and all(value == 0 for value in positions.values()):
        # Short Banknifty and go long on constituents
        positions['BankNifty'] = -calculate_shares(next_open_prices, (balance/2))
        balance += positions['BankNifty'] * next_open_prices  # Update balance
        for ticker, allocation in allocations.items():
            positions[ticker] = calculate_shares(next_open_prices_constituents[ticker], allocation)
            balance -= positions[ticker] * next_open_prices_constituents[ticker]  # Update balance
        tradebook.append(('Enter', date, positions.copy(), balance))

    elif prev_z_score is not None and (prev_z_score * current_z_score <= 0) and current_z_score<=0:
        # Square-off positions if any positions are open
        if any(value != 0 for value in positions.values()):
            # Update balance for BankNifty
            balance -= positions['BankNifty'] * next_open_prices
            # Update balance for each constituent
            for ticker in constituents:
                balance += positions[ticker] * next_open_prices_constituents[ticker]
            # Reset all positions to 0
            for key in positions.keys():
                positions[key] = 0
            tradebook.append(('Exit', date, positions.copy(), balance))

    elif current_z_score <= -2.0 and all(value == 0 for value in positions.values()):
        # Long Banknifty and short constituents
        positions['BankNifty'] = calculate_shares(next_open_prices, (balance/2))
        balance -= positions['BankNifty'] * next_open_prices  # Update balance
        for ticker, allocation in allocations.items():
            positions[ticker] = -calculate_shares(next_open_prices_constituents[ticker], allocation)
            balance += positions[ticker] * next_open_prices_constituents[ticker]  # Update balance
        tradebook.append(('Enter', date, positions.copy(), balance))

    elif prev_z_score is not None and (prev_z_score * current_z_score <= 0) and current_z_score>=0:
        # Square-off positions if any positions are open
        if any(value != 0 for value in positions.values()):
            # Update balance for BankNifty
            balance += positions['BankNifty'] * next_open_prices
            # Update balance for each constituent
            for ticker in constituents:
                balance -= positions[ticker] * next_open_prices_constituents[ticker]
            # Reset all positions to 0
            for key in positions.keys():
                positions[key] = 0
            tradebook.append(('Exit', date, positions.copy(), balance))

    # Update previous z-score for next iteration
    prev_z_score = current_z_score


last_date = z_scores.index[-1]
last_prices = {ticker: data.loc[last_date, 'Close'] if last_date in data.index else 0 for ticker, data in constituent_data.items()}
last_prices['BankNifty'] = banknifty_data.loc[last_date, 'Close'] if last_date in banknifty_data.index else 0

# Squaring-off any pending position

for key, position in positions.items():
    if position != 0:
        balance += position * last_prices[key]  # Update balance for closing position
        positions[key] = 0
tradebook.append(('Final Exit', last_date, positions.copy(), balance))

7. **Tradebook**:
   Show tradebook which must contain the entry time, exit time of positions along with entry and exit prices for banknifty as well as of the 3 constituents.



In [ ]:
final_balance = balance
performance = final_balance - initial_balance
print(f"Final Balance: {final_balance}")
print(f"Performance: {performance}")
for trade in tradebook:
    print(trade)


Final Balance: 824649.3297119141
Performance: 24649.329711914062
('Enter', Timestamp('2023-10-03 10:50:00+0530', tz='Asia/Kolkata'), {'BankNifty': 9, 'HDFCBANK.NS': -45, 'ICICIBANK.NS': -244, 'KOTAKBANK.NS': -58}, 3620.29638671875)
('Exit', Timestamp('2023-10-03 12:25:00+0530', tz='Asia/Kolkata'), {'BankNifty': 0, 'HDFCBANK.NS': 0, 'ICICIBANK.NS': 0, 'KOTAKBANK.NS': 0}, 800727.5402832031)
('Enter', Timestamp('2023-10-03 13:50:00+0530', tz='Asia/Kolkata'), {'BankNifty': -9, 'HDFCBANK.NS': 45, 'ICICIBANK.NS': 245, 'KOTAKBANK.NS': 58}, 2323.2996826171875)
('Exit', Timestamp('2023-10-03 14:25:00+0530', tz='Asia/Kolkata'), {'BankNifty': 0, 'HDFCBANK.NS': 0, 'ICICIBANK.NS': 0, 'KOTAKBANK.NS': 0}, 800375.5426025391)
('Enter', Timestamp('2023-10-04 10:00:00+0530', tz='Asia/Kolkata'), {'BankNifty': 9, 'HDFCBANK.NS': -45, 'ICICIBANK.NS': -248, 'KOTAKBANK.NS': -58}, 7305.8934326171875)
('Exit', Timestamp('2023-10-04 10:30:00+0530', tz='Asia/Kolkata'), {'BankNifty': 0, 'HDFCBANK.NS': 0, 'ICICIBANK

8. **Equity Curve**:
   Plot the equity curve of the strategy, starting with an initial balance of $10,000 (8,00,000Rs).


In [ ]:
equity_curve = pd.DataFrame(index=z_scores.index, columns=['Equity'])
equity_curve.iloc[0]['Equity'] = initial_balance

last_balance = initial_balance
current_positions = {}

for index, row in z_scores.items():
    # Update current_positions and last_balance from tradebook entries
    for trade in tradebook:
        trade_time = trade[1]
        if trade_time <= index:
            if trade[0] == 'Enter':
                # Update current_positions with the new positions from this trade
                current_positions.update(trade[2])  # Update current positions
            elif trade[0] == 'Exit':
                # Update last_balance with the value from this trade
                last_balance = trade[3]
                # Reset the positions for the tickers involved in the exit trade
                for ticker in trade[2]:
                    current_positions[ticker] = 0

    # Calculate equity based on current_positions and open prices at each interval
    equity_value = last_balance
    for ticker, position in current_positions.items():
        date = index.date()
        open_price = None
        if ticker == 'BankNifty':
            open_price = banknifty_data['Open'].get(index, None)
        else:
            open_price = constituent_data.get(ticker, {}).get('Open', {}).get(index, None)
        if open_price is not None:
            equity_value += position * open_price

    # Updating the equity value for the current index
    equity_curve.at[index, 'Equity'] = equity_value

# Ploting the equity_curve
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=equity_curve.index,
    y=equity_curve['Equity'],
    mode='lines',
    name='Equity'
))

# Adding markers for 'Enter' and 'Exit' trades
for trade in tradebook:
    trade_type, trade_time, positions, value = trade
    if trade_type == 'Enter':
        fig.add_trace(go.Scatter(
            x=[trade_time],
            y=[equity_curve.at[trade_time, 'Equity']],
            mode='markers',
            marker=dict(color='green', size=3),
            name='Enter'
        ))
    elif trade_type == 'Exit':
        fig.add_trace(go.Scatter(
            x=[trade_time],
            y=[equity_curve.at[trade_time, 'Equity']],
            mode='markers',
            marker=dict(color='red', size=3),
            name='Exit'
        ))

fig.update_layout(
    title='Equity Curve During Market Hours',
    xaxis=dict(
        title='Date and Time',
        type='category'  # This treats the x-axis values as categorical, not continuous so that it dosen't show off-market time
    ),
    yaxis_title='Equity',
    xaxis_rangeslider_visible=False
)

fig.show()

Z-Scores Over Time with entry and exit

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=z_scores.index, y=z_scores, mode='lines', name='Z-Score'))

# Function to determine if the z-score crosses zero
def is_zero_crossing(current, previous):
    return current * previous <= 0 and not pd.isna(previous)

last_marker_zero_crossing = False  # Flag to track if the last marker was a zero crossing

for idx, z_score in z_scores.items():
    previous_z_score = z_scores.shift(1).loc[idx]

    if is_zero_crossing(z_score, previous_z_score):
        # Plot zero crossing marker if the last marker was not a zero crossing
        if not last_marker_zero_crossing:
            fig.add_trace(go.Scatter(
                x=[idx],
                y=[z_score],
                mode='markers',
                marker=dict(color='blue', size=10),
                name='Z-Score Crossing Zero: Exit'
            ))
        last_marker_zero_crossing = True
    elif z_score >= 2 and previous_z_score < 2:
        # Plot entry marker for z_score >= 2
        fig.add_trace(go.Scatter(
            x=[idx],
            y=[z_score],
            mode='markers',
            marker=dict(color='green', size=10),
            name='Z-Score >= 2: Entry'
        ))
        last_marker_zero_crossing = False
    elif z_score <= -2 and previous_z_score > -2:
        # Plot entry marker for z_score <= -2
        fig.add_trace(go.Scatter(
            x=[idx],
            y=[z_score],
            mode='markers',
            marker=dict(color='red', size=10),
            name='Z-Score <= -2: Entry'
        ))
        last_marker_zero_crossing = False

fig.update_layout(
    title='Z-Scores with Entry and Exit Points',
    xaxis_title='Date',
    yaxis_title='Z-Score',
    showlegend=True
)
fig.show()


Building functions for easier management and for plotting graph between rolling regression window size vs sharp ratio and maximum drawdown.


In [ ]:
def generate_z_scores(banknifty_data, constituent_data, window_size):
    y = banknifty_data['Close']
    X = pd.DataFrame({ticker: data['Close'] for ticker, data in constituent_data.items()})
    X = sm.add_constant(X)
    # rolling regression
    model = RollingOLS(y, X, window=window_size)
    rolling_results = model.fit()

    coefficients = rolling_results.params.copy()
    predicted_values = (X * coefficients).sum(axis=1)
    residuals = y - predicted_values

    rolling_mean = residuals.rolling(window=window_size).mean()
    rolling_std = residuals.rolling(window=window_size).std()

    # Standardizing the residuals to get z-scores
    z_scores = (residuals - rolling_mean) / rolling_std
    return z_scores
generate_z_scores(banknifty_data,constituent_data,30)

Datetime
2023-10-03 09:15:00+05:30         NaN
2023-10-03 09:20:00+05:30         NaN
2023-10-03 09:25:00+05:30         NaN
2023-10-03 09:30:00+05:30         NaN
2023-10-03 09:35:00+05:30         NaN
                               ...   
2023-11-30 10:40:00+05:30   -0.686220
2023-11-30 10:45:00+05:30   -1.081640
2023-11-30 10:50:00+05:30   -1.147080
2023-11-30 10:55:00+05:30   -1.288425
2023-11-30 11:00:00+05:30   -1.342457
Length: 2947, dtype: float64

In [ ]:
def generate_tradebook(z_scores, initial_balance, constituents):
    tradebook = []
    balance = initial_balance
    positions = {ticker: 0 for ticker in constituents}

    prev_z_score = None

    # Function to calculate the number of shares to buy/sell based on allocation and stock price
    def calculate_shares(price, allocation):
        return int(allocation / price)

    for date, current_z_score in z_scores.items():
        next_open_prices = banknifty_data['Open'].get(date, None)
        next_open_prices_constituents = {ticker: constituent_data[ticker]['Open'].get(date, None) for ticker in constituents}

        if None in next_open_prices_constituents.values() or next_open_prices is None:
            continue

        total_coeff = sum(abs(coefficients[ticker][-1]) for ticker in constituents)
        allocations = {ticker: (abs(coefficients[ticker][-1]) / total_coeff) * (balance/2) for ticker in constituents}

        if current_z_score >= 2.0 and all(value == 0 for value in positions.values()):
            # Short Banknifty and go long on constituents
            positions['BankNifty'] = -calculate_shares(next_open_prices, (balance/2))
            balance += positions['BankNifty'] * next_open_prices  # Update balance
            for ticker, allocation in allocations.items():
                positions[ticker] = calculate_shares(next_open_prices_constituents[ticker], allocation)
                balance -= positions[ticker] * next_open_prices_constituents[ticker]  # Update balance
            tradebook.append(('Enter', date, positions.copy(), balance))

        elif prev_z_score is not None and (prev_z_score * current_z_score <= 0) and current_z_score<=0:
            # Square-off positions if any positions are open
            if any(value != 0 for value in positions.values()):
                # Update balance for BankNifty
                balance -= positions['BankNifty'] * next_open_prices
                # Update balance for each constituent
                for ticker in constituents:
                    balance += positions[ticker] * next_open_prices_constituents[ticker]
                # Reset all positions to 0
                for key in positions.keys():
                    positions[key] = 0
                tradebook.append(('Exit', date, positions.copy(), balance))

        elif current_z_score <= -2.0 and all(value == 0 for value in positions.values()):
            # Long Banknifty and short constituents
            positions['BankNifty'] = calculate_shares(next_open_prices, (balance/2))
            balance -= positions['BankNifty'] * next_open_prices  # Update balance
            for ticker, allocation in allocations.items():
                positions[ticker] = -calculate_shares(next_open_prices_constituents[ticker], allocation)
                balance += positions[ticker] * next_open_prices_constituents[ticker]  # Update balance
            tradebook.append(('Enter', date, positions.copy(), balance))

        elif prev_z_score is not None and (prev_z_score * current_z_score <= 0) and current_z_score>=0:
            # Square-off positions if any positions are open
            if any(value != 0 for value in positions.values()):
                # Update balance for BankNifty
                balance += positions['BankNifty'] * next_open_prices
                # Update balance for each constituent
                for ticker in constituents:
                    balance -= positions[ticker] * next_open_prices_constituents[ticker]
                # Reset all positions to 0
                for key in positions.keys():
                    positions[key] = 0
                tradebook.append(('Exit', date, positions.copy(), balance))

        # Update previous z-score for next iteration
        prev_z_score = current_z_score

    last_date = z_scores.index[-1]
    last_prices = {ticker: data.loc[last_date, 'Close'] if last_date in data.index else 0 for ticker, data in constituent_data.items()}
    last_prices['BankNifty'] = banknifty_data.loc[last_date, 'Close'] if last_date in banknifty_data.index else 0

    for key, position in positions.items():
        if position != 0:
            balance += position * last_prices[key]  # Update balance for closing position
            positions[key] = 0
    tradebook.append(('Final Exit', last_date, positions.copy(), balance))

    return tradebook
generate_tradebook(z_scores, initial_balance, constituents)


In [ ]:
def generate_equity_curve(z_scores, tradebook, initial_balance, banknifty_data, constituent_data):
    equity_curve = pd.DataFrame(index=z_scores.index, columns=['Equity'])
    equity_curve.iloc[0]['Equity'] = initial_balance

    last_balance = initial_balance
    current_positions = {}

    for index, row in z_scores.items():
        # Update current_positions and last_balance from tradebook entries
        for trade in tradebook:
            trade_time = trade[1]
            if trade_time <= index:
                if trade[0] == 'Enter':
                    # Update current_positions with the new positions from this trade
                    current_positions.update(trade[2])  # Update current positions
                elif trade[0] == 'Exit':
                    # Update last_balance with the value from this trade
                    last_balance = trade[3]
                    # Reset the positions for the tickers involved in the exit trade
                    for ticker in trade[2]:
                        current_positions[ticker] = 0

        # Calculate equity based on current_positions and open prices at each interval
        equity_value = last_balance
        for ticker, position in current_positions.items():
            date = index.date()
            open_price = None
            if ticker == 'BankNifty':
                open_price = banknifty_data['Open'].get(index, None)
            else:
                open_price = constituent_data.get(ticker, {}).get('Open', {}).get(index, None)
            if open_price is not None:
                equity_value += position * open_price

        # Update the equity value for the current index
        equity_curve.at[index, 'Equity'] = equity_value

    return equity_curve

generate_equity_curve(z_scores, tradebook, initial_balance, banknifty_data, constituent_data)

,Equity
Datetime,
2023-10-03 09:15:00+05:30,800000
2023-10-03 09:20:00+05:30,800000
2023-10-03 09:25:00+05:30,800000
2023-10-03 09:30:00+05:30,800000
2023-10-03 09:35:00+05:30,800000
...,...
2023-11-30 10:40:00+05:30,824649.329712
2023-11-30 10:45:00+05:30,824649.329712
2023-11-30 10:50:00+05:30,824649.329712


In [ ]:
def calculate_max_drawdown_and_sharpe(equity_curve):
    # Calculate daily returns
    daily_returns = equity_curve['Equity'].pct_change().dropna()

    # Sharpe Ratio without risk free rate
    sharpe_ratio = (daily_returns.mean() / daily_returns.std())

    # Maximum Drawdown
    peak  = equity_curve['Equity'].cummax()
    daily_drawdown = equity_curve['Equity'] / peak  - 1.0
    max_drawdown = daily_drawdown.min()

    return max_drawdown, sharpe_ratio


9. **Performance Metrics**:
   Calculate the sharpe ratio, maximum drawdown.
   Plot how the sharpe ratio changes with various lookback windows.

In [ ]:
calculate_max_drawdown_and_sharpe(generate_equity_curve(generate_z_scores(banknifty_data, constituent_data, window_size)
, generate_tradebook(z_scores, initial_balance, constituents), initial_balance, banknifty_data, constituent_data))


(-0.056679133153823336, 0.004537889553889074)

In [ ]:
# Initialize lists to store window sizes, max drawdowns, and sharpe ratios
window_sizes = list(range(15, 480, 5))
max_drawdowns = []
sharpe_ratios = []

# Initial balance for the tradebook
initial_balance = 800000

for window_size in window_sizes:
    z_scores = generate_z_scores(banknifty_data, constituent_data, window_size)

    tradebook = generate_tradebook(z_scores, initial_balance, constituents)

    equity_curve = generate_equity_curve(z_scores, tradebook, initial_balance, banknifty_data, constituent_data)

    # Calculate the metrics and store them
    max_dd, sharpe = calculate_max_drawdown_and_sharpe(equity_curve)
    max_drawdowns.append(max_dd)
    sharpe_ratios.append(sharpe)

# Plotting window_size vs max_drawdown and sharpe_ratio
fig = go.Figure()

# Add max_drawdown trace
fig.add_trace(go.Scatter(
    x=window_sizes,
    y=max_drawdowns,
    mode='lines+markers',
    name='Max Drawdown'
))

# Add sharpe_ratio trace
fig.add_trace(go.Scatter(
    x=window_sizes,
    y=sharpe_ratios,
    mode='lines+markers',
    name='Sharpe Ratio'
))

fig.update_layout(
    title='Window Size vs Max Drawdown and Sharpe Ratio',
    xaxis_title='Window Size (days)',
    yaxis_title='Metric Value',
    xaxis_rangeslider_visible=False
)
fig.show()